# 네이버 지도 맛집, 카페 상호명 크롤링
## 참고자료
https://youtu.be/yQ20jZwDjTE  
https://boringariel.tistory.com/5

In [7]:
! pip install selenium==4.11.2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 37.1 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: selenium
    Found existing installation: selenium 4.8.3
    Uninstalling selenium-4.8.3:
      Successfully uninstalled selenium-4.8.3


In [88]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
import time

In [10]:
service = Service()
options = webdriver.ChromeOptions()
driver = webdriver.Chrome(service=service, options=options)

In [138]:
driver.get("https://map.naver.com/v5/search/성동구 식당")

## 페이지가 다 로딩될 때까지 기다려주세요!

In [139]:
driver.switch_to.frame("searchIframe")

fnm = '' # 맨 첫번째 상호 이름입니다. 다음 페이지에서도 똑같은 상호가 나온다면, 다음 페이지가 없다고 인식하고, brk 값이 바뀌면서 반복문이 종료되지요
brk = 1
res = pd.DataFrame(columns=['storeName', 'storeType']) # 결과 파일은 판다스 데이터프레임으로 입력할겁니다

while brk: # 페이지 설정
    
    # 스크롤 끝까지 내리기
    
    view = driver.find_element(By.CSS_SELECTOR, "#_pcmap_list_scroll_container")
    SCROLL_PAUSE_TIME = 0.5

    # Get scroll height
    last_height = driver.execute_script("return document.querySelector('.Ryr1F').scrollHeight")

    while True:
        # Scroll down to bottom
        driver.execute_script("arguments[0].scrollTo(0, document.querySelector('.Ryr1F').scrollHeight);", view)

        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)

        # Calculate new scroll height and compare with last scroll height
        new_height = driver.execute_script("return document.querySelector('.Ryr1F').scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
        
    # 이름, 가게유형 가져오기 
    
    nm = driver.find_elements(By.CSS_SELECTOR, 'span.place_bluelink.TYaxT')
    storetype = driver.find_elements(By.CSS_SELECTOR, 'span.KCMnt')
    nm = list(map(lambda x: x.text, nm))
    storetype = list(map(lambda x: x.text, storetype))

        
    res = pd.concat([res,pd.DataFrame(zip(nm, storetype), columns=['storeName', 'storeType'])],axis = 0, ignore_index=True)
    res.to_csv('./res_naver.csv', index=False) # 데이터가 실시간으로 저장되도록 합니다
        
    if fnm == nm[0]:
        brk = 0
    else:
        fnm = nm[0]


    # 다음 페이지로
    driver.find_element(By.CSS_SELECTOR, "#app-root > div > div.XUrfU > div.zRM9F > a:nth-child(7) > svg").click()
    time.sleep(4) # 페이지 로딩 시간 4초

In [140]:
res

,storeName,storeType
0,오스테리아 파로,이탈리아음식
1,성수다락,"스파게티,파스타전문"
2,타코튜즈데이 성수 본점,"멕시코,남미음식"
3,요쇼쿠 성수,일식당
4,술있는식탁 성수본점,요리주점
...,...,...
345,리타르단도,양식
346,스시오오모토,일식당
347,성수일구,일식당
348,오와리 뚝섬점,이자카야
